# Black Body Spectrum

The radiance spectrum of a [Black-body](https://en.wikipedia.org/wiki/Black_body) can
be found using [Plank's law](https://en.wikipedia.org/wiki/Planck%27s_law#Different_forms):

$L_{\lambda}(T,\lambda) = \frac{2 h c^2}{\lambda^5} \frac{1}{\exp{\frac{hc}{k_{B}T\lambda}}-1} \quad \left[\mathrm{\frac{W}{sr \cdot m^2 \cdot m}}\right]$

$L_{\nu}(T,\nu) = \frac{2 h \nu^3}{c^2} \frac{1}{\exp{\frac{h\nu}{k_{B}T}}-1} \quad \left[\mathrm{\frac{W}{sr \cdot m^2 \cdot Hz}}\right]$

with:

- $T$ = temperature in Kelvin
- $h$ = $6.62607015 \times 10^{-34} \mathrm{\frac{J}{Hz}}$, the Planck constant
- $c$ = $299792458 \mathrm{\frac{m}{s}}$, the speed of light
- $k_B$ = $1.380649 \times 10^{-23} \mathrm{\frac{J}{K}}$, the Boltzmann constant
- $\lambda$ = the wavelength of light
- $\nu$ = $\frac{c}{\lambda}$, the frequency of light

Notice how $L_{\lambda}(T,\lambda) = \frac{c}{\lambda^2} L_{\nu}(T,\nu)$ because one is [expressed in terms per $d\lambda$, while the other per $d\nu$](https://en.wikipedia.org/wiki/Planck%27s_law#Correspondence_between_spectral_variable_forms).

**Here, we continue with $L_{\lambda}(T,\lambda)$**.

The wavelength of the peak value can be found using [Wien's displacement law](https://en.wikipedia.org/wiki/Black-body_radiation#Wien's_displacement_law):

$\lambda_\text{peak} = \frac{b}{T}$

with:

- $b$ = $2.897771955 \times 10^{-3} \mathrm{\frac{m}{K}}$, Wien's displacement constant

References:

- https://en.wikipedia.org/wiki/Black-body_radiation
- https://phys.libretexts.org/Bookshelves/University_Physics/Book%3A_University_Physics_(OpenStax)/University_Physics_III_-_Optics_and_Modern_Physics_(OpenStax)/06%3A_Photons_and_Matter_Waves/6.02%3A_Blackbody_Radiation
- https://physics.info/planck/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

h = 6.62607015e-34
c = 299792458
kB = 1.380649e-23
b = 2.897771955e-3


def radiance(temperature, wavelength):
    return (
        2
        * h
        * c**2
        / (wavelength**5 * (np.exp(h * c / (kB * temperature * wavelength)) - 1))
    )


def irradiance(temperature, wavelength):
    return 4 * np.pi * radiance(temperature, wavelength)


def peakWavelength(temperature):
    return b / temperature


temperatures = [3000, 4000, 5000, 6000, 6500, 7000]
wavelengths = np.arange(200e-9, 1000e-9, 10e-9)

peakWavelengths = np.array([peakWavelength(T) for T in temperatures])
peakValues = np.array([irradiance(T, peakWavelength(T)) for T in temperatures])

plt.plot(
    peakWavelengths,
    peakValues / 1e12,
    label=f"peak",
    color="black",
    marker="o",
    linestyle="dashed",
)
for T in temperatures:
    plt.plot(wavelengths, irradiance(T, wavelengths) / 1e12, label=f"{T}K")
plt.xlabel("Wavelength")
plt.ylabel("Irradiance [kW/m^2/nm]")
plt.legend()
plt.show()

## Normalized Spectra

The `BlackBody` spectrum in `liar` is normalized so that the peak values are always one.

In [ ]:
from matplotlib.colors import TABLEAU_COLORS

import liar

observer = liar.Observer.standard()
wavelengths2 = np.array(observer.wavelengths)

spectra = [liar.spectra.BlackBody(T) for T in temperatures]
peakWavelengths = np.array([s.peakWavelength for s in spectra])
peakValues = np.array([s(w) for s, w in zip(spectra, peakWavelengths)])
colors = list(TABLEAU_COLORS)[: len(temperatures)]

plt.plot(
    peakWavelengths,
    peakValues,
    label=f"peak",
    color="black",
    marker="o",
    linestyle="dashed",
)
for T, s, color in zip(temperatures, spectra, colors):
    L = radiance(T, wavelengths)
    L /= np.max(L)
    plt.plot(wavelengths, L, label=f"{T}K", color=color, linestyle="dotted")

    Ls = np.array([s(w) for w in wavelengths2])
    plt.plot(wavelengths2, Ls, label=f"{T}K", color=color)
plt.xlabel("Wavelength")
plt.ylabel("Irradiance [kW/m^2/nm]")
plt.legend()
plt.show()

The `luminance` values of these spectra will be less than one:

In [ ]:
temps = np.arange(1000, 10000, 200)
plt.plot(
    temps,
    [liar.spectra.BlackBody(float(T)).luminance for T in temps],
    label=f"luminance",
    color="black",
)
plt.xlabel("Temperature [K]")
plt.ylabel("Luminance [cd/m^2]")
plt.legend()
plt.show()

for T, s in zip(temperatures, spectra):
    print(f"{T}K: y={s.luminance}, peak={s.peakWavelength / 1e-9}nm")

Maximum luminance is achieved at temperature 5200K

In [ ]:
s = liar.spectra.BlackBody(5200)
print(f"5200K: y={s.luminance}, peak={s.peakWavelength / 1e-9}nm")